# FIT5196 Assessment 1 - EDA

Due Date: 23:55, Sunday, 14 September 2025


---



#### Group 35:
Member 1: Adrian Leong Tat Wei, (27030768), atleo4@student.monash.edu, Contribution

Member 2: Jun Yuan, (35833645), jyua0050@student.monash.edu, Contribution

...

---

### Table of Content

1.
2.
3.



In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.9 MB/s eta 0:00:00


## 1. Load, parse and merge data files






In [ ]:
import pandas as pd
import re
import emoji

base = "./drive/MyDrive/FIT5196/Assignment1/"

### 1.1 Load data files

In [ ]:
# https://docs.python.org/3/library/xml.etree.elementtree.html
import xml.etree.ElementTree as ET

# Parse the XML file
tree = ET.parse(base + "Group035.xml")
root = tree.getroot()

# root is <FlickrData>, iterate over each <Record>
records = []
for record in root.findall("Record"):
    record_dict = {child.tag: child.text for child in record}
    records.append(record_dict)

print(type(records))   # <class 'list'>
print(records[0])      # print the first record

# Assuming records (from XML) is loaded as a list of dicts
df_xml = pd.DataFrame(records)


<class 'list'>
{'PostID': '49219111338.0', 'UserID': '124930081@N08', 'secret': '1187640507', 'server': '65535.0', 'title': 'DSC_0652 National Trust Museum (former Port Pirie Railway Station), 73-77 Ellen Street, Port Pirie, South Australia', 'ispublic': '1.0', 'isfriend': '0.0', 'isfamily': '0.0', 'farm': '66.0', 'City': 'Adelaide', 'Country': 'Australia', 'Post_date': '2019-12-14 22:49:28', 'Taken_date': '2019-09-18 13:15:16', 'tags': 'portpirie,museum,railwaystation,southaustralia,australia,architecture,heritage,historic,', 'latitude': '-33.175428', 'longitude': '138.010339', 'description': 'Port Pirie station was the original station in Port Pirie. It opened in 1875 when the Port Pirie-Cockburn line opened to Gladstone. The original building was replaced in 1902.\n\nState Heritage ID: 10229', 'min_taken_date': '2019-09-18 00:00:00'}


In [ ]:
# https://www.geeksforgeeks.org/python/read-json-file-using-python/
import json

# Open and load the JSON file
with open(base + "Group035.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

# json_data is now a list of dictionaries
print(type(json_data))  # <class 'list'>
print(json_data[0])     # print the first item

# Assuming json_data is loaded as a list of dicts
df_json = pd.DataFrame(json_data)


<class 'list'>
{'PostID': 51424269137.0, 'UserID': '73561613@N06', 'secret': '9aa2d9ed0e', 'server': 65535.0, 'title': 'Aerial view at the beach with waves', 'ispublic': 1.0, 'isfriend': 0.0, 'isfamily': 0.0, 'farm': 66.0, 'City': 'Woy Woy', 'Country': 'Australia', 'Post_date': '2021-09-05 01:22:55', 'Taken_date': '2021-08-26 09:29:57', 'tags': 'swell,landscape,winter,nature,water,sky,surf,windy,aerial,waves,newsouthwales,sea,uminabeach,morning,blue,beach,ocean,australia,coast,earlymorning,coastal,nsw,outdoors,waterscape,seascape,centralcoast,southerlyswell,seaside,', 'latitude': -33.527998, 'longitude': 151.315008, 'description': 'Southerly swell producing waves at Umina Beach on the Central Coast, NSW, Australia.', 'min_taken_date': '2021-08-26 00:00:00'}


### 1.2 Merge dataset

In [ ]:
# -------------------------------
# Clean and convert data types
# -------------------------------
# Specify numeric columns
numeric_cols = ["PostID", "UserID", "server", "ispublic", "isfriend", "isfamily", "farm", "latitude", "longitude"]
for col in numeric_cols:
    for df in [df_json, df_xml]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # converts invalid entries to NaN
            if df[col] == "null" or df[col] == None or df[col] == "":
                df[col] = "NaN"


# Specify datetime columns
date_cols = ["Post_date", "Taken_date", "min_taken_date"]
for col in date_cols:
    for df in [df_json, df_xml]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            if df[col] == "null" or df[col] == None or df[col] == "":
                df[col] = "NaN"


# Specify text (alphanumeric) columns
text_cols = ["title", "tags", "description", "City", "Country"]
for col in text_cols:
    for df in [df_json, df_xml]:
        if col in df.columns:
            # TODO: convert to lower case
            if df[col] == "null" or df[col] == None or df[col] == "":
                df[col] = "NaN"


# Last column not to be cleaned: "secret", is alphanumeric, unsure if it should be cleaned
# TODO: No XML or JSON tags, using regular expressions
# TODO: No non-english characters, using regular expressions

# # Fill missing values if desired
# fill_values = {"City": "Unknown", "Country": "Unknown"}
# for col, val in fill_values.items():
#     for df in [df_json, df_xml]:
#         if col in df.columns:
#             df[col] = df[col].fillna(val)

# # Add source column (source column must not be in final output, only for debug)
# df_json["source"] = "json"
# df_xml["source"] = "xml"

# -------------------------------
# Merge/Combine DataFrames
# -------------------------------

# Option 1: Concatenate all posts, keeping all users
df_all = pd.concat([df_json, df_xml], ignore_index=True)
# df_all.rename(columns={"PostID": "Post_ID", "UserID": "User_ID"})

# Option 2: Merge by UserID, keeping only users present in both
# df_merged = pd.merge(df_json, df_xml, on="UserID", suffixes=("_json", "_xml"))


# -------------------------------
# 5. Inspect final DataFrame
# -------------------------------
# print(df_all.info())
# print(df_all.head())

In [ ]:
# Data cleaning and type conversion

def clean_text_content(text_input):
    """
    Clean text fields according to assignment specifications:
    - Convert to lowercase (except NaN)
    - Remove XML/JSON tags using regex
    - Remove emojis using emoji library
    - Remove non-English characters using regex
    - Replace null values with 'NaN'
    """
    # Handle null/empty values first
    if pd.isna(text_input) or text_input is None or str(text_input).strip() == '' or text_input == "null":
        return 'NaN'

    text_str = str(text_input)

    # Skip conversion for 'NaN' strings
    if text_str == 'NaN':
        return 'NaN'

    # Convert to lowercase as required
    text_str = text_str.lower()

    # Remove XML/JSON tags - matches <...> patterns
    text_str = re.sub(r'<[^>]*>', '', text_str)
    # Remove HTML entities like &quot; &amp; etc
    text_str = re.sub(r'&[a-zA-Z0-9#]+;', '', text_str)

    # Remove emoji characters using emoji library
    text_str = emoji.replace_emoji(text_str, replace='')

    # Keep only valid characters, numbers and basic punctuation
    text_str = re.sub(r'[^a-zA-Z0-9\s.,!?;:()\-\'"/\\@#$%&*+=<>~`|{}[\]^]+', '', text_str)

    # Clean up multiple spaces
    text_str = re.sub(r'\s+', ' ', text_str)
    text_str = text_str.strip()

    # Return NaN if empty after cleaning
    if len(text_str) == 0:
        return 'NaN'

    return text_str

# Handle numeric columns
numeric_columns = ["PostID", "UserID", "server", "ispublic", "isfriend", "isfamily", "farm", "latitude", "longitude"]
for col in numeric_columns:
    for dataframe in [df_json, df_xml]:
        if col in dataframe.columns:
            dataframe[col] = pd.to_numeric(dataframe[col], errors='coerce')

# Handle datetime columns
datetime_columns = ["Post_date", "Taken_date", "min_taken_date"]
for col in datetime_columns:
    for dataframe in [df_json, df_xml]:
        if col in dataframe.columns:
            dataframe[col] = pd.to_datetime(dataframe[col], errors='coerce')


text_columns_to_clean = ["title", "tags", "description", "City", "Country"]
for col in text_columns_to_clean:
    for dataframe in [df_json, df_xml]:
        if col in dataframe.columns:
            print(f"Cleaning {col} field...")
            dataframe[col] = dataframe[col].apply(clean_text_content)

print("Text field cleaning completed successfully")

# Merge datasets
# -------------------------------
df_all = pd.concat([df_json, df_xml], ignore_index=True)

print(f"Merged dataset contains {len(df_all)} records from both JSON and XML files")
print(f"Dataset has {len(df_all.columns)} columns total")

print("\nSample of cleaned text fields:")
for i in range(min(3, len(df_all))):
    print(f"Record {i+1}:")
    if 'title' in df_all.columns:
        title_sample = df_all.iloc[i]['title']
        print(f"  Title: {title_sample[:80]}..." if len(str(title_sample)) > 80 else f"  Title: {title_sample}")
    if 'description' in df_all.columns:
        desc_sample = df_all.iloc[i]['description']
        print(f"  Description: {desc_sample[:80]}..." if len(str(desc_sample)) > 80 else f"  Description: {desc_sample}")
    if 'tags' in df_all.columns:
        tags_sample = df_all.iloc[i]['tags']
        print(f"  Tags: {tags_sample}")
    print()


Cleaning title field...
Cleaning title field...
Cleaning tags field...
Cleaning tags field...
Cleaning description field...
Cleaning description field...
Cleaning City field...
Cleaning City field...
Cleaning Country field...
Cleaning Country field...
Text field cleaning completed successfully
Merged dataset contains 70000 records from both JSON and XML files
Dataset has 18 columns total

Sample of cleaned text fields:
Record 1:
  Title: aerial view at the beach with waves
  Description: southerly swell producing waves at umina beach on the central coast, nsw, austra...
  Tags: swell,landscape,winter,nature,water,sky,surf,windy,aerial,waves,newsouthwales,sea,uminabeach,morning,blue,beach,ocean,australia,coast,earlymorning,coastal,nsw,outdoors,waterscape,seascape,centralcoast,southerlyswell,seaside,

Record 2:
  Title: mack muster 2020
  Description: rowlspec titan
  Tags: NaN

Record 3:
  Title: NaN
  Description: NaN
  Tags: melbourne,victoria,australia,urban,city,cbd,walk,outdoor,win

In [ ]:
import re
import wordninja

def rename_column(colname: str) -> str:
    """
    Convert column names into Title_Case with underscores.
    Handles camelCase, PascalCase, acronyms, and concatenated words.
    """
    # Step 1: Split camelCase / PascalCase into separate words
    # e.g. UserID -> ['User', 'ID'], isPublic -> ['is', 'Public']
    camel_split = re.sub(r'([a-z])([A-Z])', r'\1 \2', colname)

    # Step 2: Split on underscores (already separated words)
    tokens = re.split(r'[_\s]+', camel_split)

    final_tokens = []
    for token in tokens:
        if not token:
            continue
        # Step 3: Preserve acronyms (all caps, length > 1)
        if token.isupper() and len(token) > 1:
            final_tokens.append(token)
        else:
            # Step 4: Word segmentation for lowercase tokens
            if token.islower():
                split_words = wordninja.split(token)
            else:
                split_words = [token]
            # Step 5: Capitalize first letter of each segment
            final_tokens.extend([w.capitalize() for w in split_words])

    # Step 6: Join with underscores
    return "_".join(final_tokens)

# Example usage on your dataframe:
df_all.rename(columns=lambda c: rename_column(c), inplace=True)

print(df_all.columns)

In [ ]:
df_all.to_csv('Group035_dataset.csv', index=False)

## 2. EDA

### 2.1 Dataset overview

### 2.2 Univariate analysis

### 2.3 Bivariate analysis

### 2.4 Multivariate analysis

## 3. Key insights and research questions

### 3.1 Key findings

### 3.2 Machine Learning research questions and justification

## Reference

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


After you run the cell above and authorize Google Drive access, your Drive will be mounted at `/content/drive`. You can then access your files using the path `/content/drive/MyDrive/your_folder_name/your_file_name`.

Now, I'll update the code cells to load the XML and JSON files from your Google Drive. **Please update the file paths in the code cells below to the correct location of your files in Google Drive.**

In [ ]:
import os
print(os.listdir('.'))

['.config', 'drive', 'sample_data']
